# Metrics and Loss Functions from First Principles

This is a series of math function recreations for practice.

<br>
<br>

In [1]:
import numpy as np
from icecream import ic
import torch
from torch import Tensor

<br>

# Mean Squared Error
AKA **L2** Loss

Notes:
1. Tends to exaggerate outliers (nice for interpretation as a metric, but...)
2. On it's own, it can be poor as a loss function because it will adjust models based on exaggerated noise.
3. You should interpret it by it's shape: If the loss is high, MSE will report it *very* high
4. Linearize it by taking the square root for interpretability and removal of exaggeration of noise/outliers.

<br>

$$
\large\begin{equation}
MAE = \frac{1}{n}\sum_{i=1}^{n}(y_i - \widehat{y_i})^2  \\\\
\end{equation}
$$

<br>

$$

\small\begin{aligned}
Where, \\
\widehat{y_i} &- i^{th}\, predicted\, output \\
y_i &- i^{th}\, actual\, output \\
n &- number\, of\, observations\, in\, sample\, or\, batch \\
\end{aligned}
$$

In [2]:
def mse (yhat:Tensor, y:Tensor)->Tensor:   
    assert(yhat.size() == y.size()), 'yhat and y need to be the same size'
    assert(yhat.size != 0), 'yhat and y cannot be empty'

    n = yhat.size()[0]
    inner = (yhat - y)**2
    return sum(inner) / n
    

<br>
<br>

# Mean Absolute Error
AKA **L1** Loss

$$
\large\begin{equation}
MAE = \frac{1}{n}\sum_{i=1}^{n}\mid y_i - \widehat{y_i}\mid  \\\\
\end{equation}
$$

<br>

$$

\small\begin{aligned}
Where, \\
\widehat{y_i} &- i^{th}\, predicted\, output \\
y_i &- i^{th}\, actual\, output \\
n &- number\, of\, observations\, in\, sample\, or\, batch \\
\end{aligned}
$$

In [8]:
def mae (yhat, y):
    
    assert(len(yhat) == len(y))
    assert(len(yhat) != 0)
    
    # n = len(yhat)
    # inner = [np.abs(yi - yhati) for yi, yhati in zip(yhat, y)]
    # return sum(inner) / n

    return torch.sum(np.abs(yhat - y)) / len(yhat)

<br>
<br>

# Huber Loss / Metric
**Smooth L1** Loss with an adjustable cutoff $(\delta)$.

reference: <a href='https://pytorch.org/docs/stable/generated/torch.nn.SmoothL1Loss.html'>https://pytorch.org/docs/stable/generated/torch.nn.SmoothL1Loss.html</a>

<br>

Huber is basically a combination of L1 and L2 loss functions.  As our loss approaches the minimum, it will use the MSE and as the loss increases, it will switch over to the MAE - thus kind of giving the best of both worlds.


$$
\large\begin{equation}\begin{align*}
Huber \quad & = L_\delta(y, \widehat{y}) 
& = \begin{cases}
\:\frac{1}{2}(y - \widehat{y})^2 \quad\quad\quad\quad &for \;\lvert y-\widehat{y}\rvert \,\le\, \delta \\
\:\delta\left(\lvert y - \widehat{y}\rvert - \frac{1}{2}\delta\right) \quad\quad &for \;\lvert y-\widehat{y}\rvert \,\gt\, \delta
\end{cases}
\end{align*}
\end{equation}
$$

<br>

$$

\small\begin{aligned}
Where, \\
\widehat{y_i} &- i^{th}\, predicted\, output \\
y_i &- i^{th}\, actual\, output \\
n &- number\, of\, observations\, in\, sample\, or\, batch \\
\end{aligned}
$$

In [4]:
# @TODO: Huber has only just been implemented in PyTorch 1.9 so there's nothing to compare against.
# I tried getting it to match torch.nn.SmoothL1Loss with beta=1.0 but it doesn't match.
# Once I upgrade to Pytorch=1.9, I'll add this back in.

def huber(yhat:Tensor, y:Tensor, delta=1.0) -> Tensor:
    
    assert(len(yhat) == len(y))
    assert(len(yhat)) != 0
    
    residual = y-yhat
    
    if all(torch.abs(residual) < delta):
        return torch.mean(
            0.5 * (residual)**2
        )
   
    else:  # in this case,  abs(y-yhat >= delta)
        return torch.mean(
            delta * (torch.abs(residual) - 0.5 * delta)
        )

<br>
<br>
<br>

# Unit Tests

In [7]:

from numpy.random import default_rng
from sklearn.metrics import mean_squared_error, mean_absolute_error
import unittest
from torch.nn import MSELoss, SmoothL1Loss #, HuberLoss

class TestEntireNotebook(unittest.TestCase):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.rng = default_rng()
    
    # def test_mse(self):
    #     # Generate random Lists
    #     yhat = self.rng.standard_normal(1000)
    #     y = self.rng.standard_normal(1000)

    #     for i in range(100):
    #         self.assertEqual(
    #             np.round(mse(yhat,y),5), 
    #             np.round(mean_squared_error(yhat,y),5)
    #         )
            
    def test_mse_torch(self):
        loss = MSELoss()
        # yhat = torch.tensor(self.rng.standard_normal(1000))
        # y = torch.tensor(self.rng.standard_normal(1000))
        yhat = torch.randn(1000)
        y = torch.randn(1000)

        for i in range(100):
            self.assertAlmostEqual(  # reconvert to numpy array first
                mse(yhat,y).numpy().astype(np.float32), 
                loss(yhat,y).numpy().astype(np.float32),
                places=5
            )        
        
    # def test_huber_torch(self):
    #     loss = SmoothL1Loss()
    #     yhat = torch.randn(1000)
    #     y = torch.randn(1000)

    #     ic(loss(yhat,y))
    #     ic(huber(yhat,y))
    #     for i in range(100):
    #         self.assertAlmostEqual(  # reconvert to numpy array first
    #             huber(yhat,y).numpy().astype(np.float32), 
    #             loss(yhat,y).numpy().astype(np.float32)
    #         )    

    def test_mae(self):
        # Generate random Lists
        rng = default_rng()
        yhat = torch.randn(1000)
        y = torch.randn(1000)

        for i in range(100):
            self.assertAlmostEqual(
                mae(yhat,y), 
                mean_absolute_error(yhat,y),
                places=6
            )
    
# Run the unit tests
unittest.main(argv=[''], verbosity=2, exit=False)

test_mae (__main__.TestEntireNotebook) ... ERROR
test_mse_torch (__main__.TestEntireNotebook) ... ok

ERROR: test_mae (__main__.TestEntireNotebook)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-7-30941090972b>", line 57, in test_mae
    mae(yhat,y),
  File "<ipython-input-3-9f7986d075fc>", line 10, in mae
    return torch.sum(np.abs(yhat - y)) / yhat.size
TypeError: unsupported operand type(s) for /: 'Tensor' and 'builtin_function_or_method'

----------------------------------------------------------------------
Ran 2 tests in 0.202s

FAILED (errors=1)


In [6]:
# import time
# rng = np.random.default_rng()

# l1 = rng.standard_normal(10000000)
# l2 = rng.standard_normal(10000000)

# # l1=list(l1)
# # l2=list(l2)
# start_time = time.time()

# l3 = np.subtract(l1,l2)

# print("Process finished --- %s seconds ---" % (time.time() - start_time))